In [1]:
import gym
import numpy as np
import tensorflow as tf
from tensorflow.contrib.layers import flatten, conv2d, fully_connected
from collections import deque, Counter
import random
from datetime import datetime
from IPython import display
import matplotlib.pyplot as plt
import time
from hide_and_seek_3thief import HideAndSeekEnv
%matplotlib inline

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# 환경 설정
env = HideAndSeekEnv()
n_outputs = env.action_space.n
env.render()

o - - - - - - - - - - - - - - - - - o
| P |       |           |           |
|           | - |   |   | - | - | 3 |
|   |               |   |         2 |
| - | 2                     | - |   |
|       |   |                       |
|   |                       | - |   |
|   |                       |       |
|   | - | - | - |           | - |   |
|                               | - |
o - - - - - - - - - - - - - - - - - o



In [3]:
# Q 네트워크를 구축하기 위해 q_network라는 함수를 정의
# Q 네트워크를 입력하고 해당 상태의 모든 작업에 대한 Q 값을 얻는다.
# fully connected layer가 이어지는 동일한 패딩을 가진 3개의 convolutional layers로 Q 네트워크를 구축
tf.reset_default_graph()

def q_network(X, name_scope):
    #layers 초기화
    initializer = tf.contrib.layers.variance_scaling_initializer()
    
    with tf.variable_scope(name_scope) as scope:
        #convolutional layers 초기화
        layer_1 = conv2d(X, num_outputs=32, kernel_size=(5,5), stride=4, padding="SAME", weights_initializer=initializer)
        tf.summary.histogram('layer_1',layer_1)
        
        layer_2 = conv2d(layer_1, num_outputs=64, kernel_size=(3,3), stride=2, padding="SAME", weights_initializer=initializer)
        tf.summary.histogram('layer_2',layer_2)
        
        layer_3 = conv2d(layer_2, num_outputs=64, kernel_size=(3,3), stride=1, padding="SAME", weights_initializer=initializer)
        tf.summary.histogram('layer_3',layer_3)
        
        # fully connected layer에 공급하기 전에 layer_3의 결과를 평탄화
        flat = flatten(layer_3)
        
        fc = fully_connected(flat, num_outputs=128, weights_initializer=initializer)
        tf.summary.histogram('fc',fc)
        
        output = fully_connected(fc, num_outputs=n_outputs, activation_fn=None, weights_initializer=initializer)
        tf.summary.histogram('output',output)
        
        #vars는 가중치와 같은 네트워크 매개변수를 저장
        vars = {v.name[len(scope.name):]: v for v in tf.get_collection(key=tf.GraphKeys.TRAINABLE_VARIABLES, scope=scope.name)}
        return vars, output

In [4]:
# 엡실론 그리디 정책을 수행하기 위해 epsilon_greedy라는 함수를 정의
# 영원히 탐색하고 싶지 않기 때문에 엡실론의 가치가 시간이 지남에 따라 쇠퇴하는 쇠퇴 엡실론 탐욕 정책을 사용
# 즉, 시간이 지남에 따라 우리 정책은 좋은 행동만 이용할 것입니다.
eps_min=0.05
eps_max=0.5
eps_decay_steps = 500000

def epsilon_greedy(action, step):
    p = np.random.random(1).squeeze()
    epsilon = max(eps_min, eps_max - (eps_max-eps_min) * step/eps_decay_steps)
    if np.random.rand() < epsilon:
        return np.random.randint(n_outputs)
    else:
        return action

In [5]:
# 경험을 보유하는 50000의 경험 버퍼를 초기화
# 에이전트의 모든 경험, 즉 (상태, 행동, 보상)을 경험 버퍼에 저장하고 네트워크 훈련을 위해 이 경험의 미니배치에서 샘플링
buffer_len = 20000
exp_buffer = deque(maxlen=buffer_len)

# 메모리에서 경험을 샘플링하기 위해 sampled_memories라는 함수를 정의
# 배치 크기는 메모리에서 샘플링된 경험의 수이다.
def sample_memories(batch_size):
    perm_batch = np.random.permutation(len(exp_buffer))[:batch_size]
    mem = np.array(exp_buffer)[perm_batch]
    return mem[:,0],mem[:,1],mem[:,2],mem[:,3],mem[:,4]

In [6]:
# 하이퍼파라미터 정의
num_episodes = 1000
batch_size = 48
input_shape = (None, 11, 19, 1)
learning_rate = 0.001
X_shape = (None, 11, 19, 1)
discount_factor = 0.97

global_step = 0
copy_steps = 100
steps_train = 4
start_steps = 2000

In [7]:
logdir = 'hideandseek_logs'
tf.reset_default_graph()

# 입력에 대한 게임 상태를 정의
X = tf.placeholder(tf.float32, shape=X_shape)

# 교육을 토글하기 위해 in_training_model이라는 부울을 정의
in_training_mode = tf.placeholder(tf.bool)

In [8]:
# 기본 대상 Q 네트워크를 구축
# 입력 X를 취하고 상태의 모든 작업에 대해 Q 값을 생성하는 Q 네트워크를 구축
mainQ, mainQ_outputs = q_network(X, 'mainQ')

# 목표 Q 네트워크를 구축
targetQ, targetQ_outputs = q_network(X, 'targetQ')

In [9]:
# 행동 값에 대한 자리 표시자 정의
X_action = tf.placeholder(tf.int32, shape=(None,))
Q_action = tf.reduce_sum(targetQ_outputs * tf.one_hot(X_action, n_outputs), axis=-1,keep_dims=True)

In [10]:
# 기본 Q 네트워크 매개변수를 대상 Q 네트워크에 복사
copy_op = [tf.assign(main_name, targetQ[var_name]) for var_name, main_name in mainQ.items()]
copy_target_to_main = tf.group(*copy_op)

In [11]:
# gradient descent optimizer를 사용하여 손실 계산 및 최적화

# 행동에 대한 자리 표시자를 정의
y = tf.placeholder(tf.float32, shape=(None,1))

# 실제 값과 예측 값의 차이인 손실을 계산
loss = tf.reduce_mean(tf.square(y - Q_action))

# loss을 최소화하기 위해 adam optimizer를 사용
optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()

loss_summary = tf.summary.scalar('Loss', loss)
merge_summary = tf.summary.merge_all()
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [12]:
# tensorflow와 그 안의 모델을 실행
with tf.Session() as sess:
    init.run()
    
    # 로그를 기록할 파일 열기
    log_file = "hideandseek_log.txt"
    
    #에피소드
    for i in range(num_episodes):
        done = False
        obs = env.reset()
        epoch = 0
        episodic_reward = 0
        actions_counter = Counter()
        episodic_loss = []
        
        #상태가 최종 상태가 아닌 동안
        while not done:
            # 전처리된 게임 화면 가져오기
            obs = env.render('dqn')
            
            # 게임 화면을 피드하고 각 작업에 대한 Q 값을 가져오기
            actions = mainQ_outputs.eval(feed_dict={X:[obs], in_training_mode: False})
            
            # 행동 가져오기
            action = np.argmax(actions, axis =-1)
            actions_counter[str(action)] += 1
            
            #엡실론 그리디 정책을 사용하여 행동 선택
            action = epsilon_greedy(action, global_step)
            
            #행동을 수행하고 다음 상태인 next_obs로 이동하여 보상을 받는다.
            next_obs, reward, done, _ = env.step(action)
            
            #이 전환을 재생 버퍼에 경험으로 저장
            exp_buffer.append([obs, action, next_obs, reward, done])
            
            #특정 단계 후에 경험 버퍼의 샘플로 Q 네트워크를 훈련
            if global_step % steps_train == 0 and global_step > start_steps:
                # 샘플 경험
                o_obs, o_act, o_next_obs, o_rew, o_done = sample_memories(batch_size)
                
                # 상태
                o_obs = [x for x in o_obs]
                
                # 다음 상태
                o_next_obs = [x for x in o_next_obs]
                
                # 다음 행동
                next_act = mainQ_outputs.eval(feed_dict={X:o_next_obs, in_training_mode:False})
                
                # 보상
                y_batch = o_rew + discount_factor * np.max(next_act, axis=-1) * (1-o_done)
                
                # 모든 요약을 병합하고 파일에 쓰기
                mrg_summary = merge_summary.eval(feed_dict={X: o_obs, y:np.expand_dims(y_batch, axis=-1), X_action:o_act, in_training_mode:False})
                file_writer.add_summary(mrg_summary, global_step)
                
                # 네트워크 훈련 및 loss 계산
                train_loss, _ = sess.run([loss, training_op], feed_dict={X:o_obs, y:np.expand_dims(y_batch, axis=-1), X_action:o_act, in_training_mode: True})
                episodic_loss.append(train_loss)
                
            # 일정 간격 후에 주요 Q 네트워크 가중치를 대상 Q 네트워크에 복사
            if (global_step+1)%copy_steps == 0 and global_step > start_steps:
                copy_target_to_main.run()
                    
            epoch += 1
            global_step += 1
            episodic_reward += reward
            
        # 파일에 로그 기록
        with open(log_file, "a") as file:
            file.write("Epoch: " + str(epoch) + ", Reward: " + str(episodic_reward))
            file.write(", Loss: " + str(np.mean(episodic_loss)))
            file.write(", Actions: " + str(actions_counter) + "\n")
        
        # 정보 출력
        print('Epoch', epoch, 'Reward', episodic_reward)
        
    # 저장 객체 생성
    saver = tf.train.Saver()
    
    # 모델 저장
    saver.save(sess, './model', global_step=global_step)

Epoch 2159 Reward -15590
Epoch 644 Reward -3950
Epoch 1431 Reward -7257
Epoch 2535 Reward -13950
Epoch 1039 Reward -4588
Epoch 720 Reward -2307
Epoch 1009 Reward -3244
Epoch 1467 Reward -5313
Epoch 1749 Reward -5829
Epoch 1063 Reward -3055
Epoch 591 Reward -1701
Epoch 566 Reward -1631
Epoch 2884 Reward -9259
Epoch 1214 Reward -3513
Epoch 911 Reward -3281
Epoch 803 Reward -2723
Epoch 149 Reward -116
Epoch 355 Reward -889
Epoch 676 Reward -1813
Epoch 273 Reward -645
Epoch 228 Reward -645
Epoch 591 Reward -1440
Epoch 84 Reward -33
Epoch 660 Reward -1986
Epoch 1254 Reward -4290
Epoch 334 Reward -661
Epoch 2473 Reward -7714
Epoch 801 Reward -2010
Epoch 2190 Reward -6495
Epoch 636 Reward -1944
Epoch 738 Reward -1902
Epoch 424 Reward -1093
Epoch 1859 Reward -5903
Epoch 606 Reward -1807
Epoch 2227 Reward -6838
Epoch 613 Reward -1354
Epoch 1234 Reward -3028
Epoch 1427 Reward -3653
Epoch 602 Reward -1352
Epoch 432 Reward -1056
Epoch 2130 Reward -6057
Epoch 327 Reward -493
Epoch 3339 Reward -1050

Epoch 351 Reward -588
Epoch 48 Reward 164
Epoch 97 Reward -46
Epoch 278 Reward -524
Epoch 260 Reward -191
Epoch 643 Reward -1195
Epoch 144 Reward -318
Epoch 148 Reward -88
Epoch 657 Reward -2406
Epoch 961 Reward -1900
Epoch 857 Reward -3344
Epoch 172 Reward -391
Epoch 66 Reward 191
Epoch 534 Reward -1374
Epoch 379 Reward -1219
Epoch 1269 Reward -4530
Epoch 147 Reward -141
Epoch 278 Reward -632
Epoch 373 Reward -988
Epoch 140 Reward -143
Epoch 655 Reward -1198
Epoch 416 Reward -1139
Epoch 314 Reward -543
Epoch 220 Reward -115
Epoch 563 Reward -953
Epoch 296 Reward -344
Epoch 394 Reward -586
Epoch 329 Reward -494
Epoch 545 Reward -899
Epoch 1010 Reward -2588
Epoch 1327 Reward -3616
Epoch 667 Reward -1858
Epoch 232 Reward -694
Epoch 1683 Reward -3945
Epoch 171 Reward -444
Epoch 190 Reward -373
Epoch 489 Reward -888
Epoch 217 Reward -409
Epoch 820 Reward -2281
Epoch 258 Reward -315
Epoch 521 Reward -911
Epoch 453 Reward -1671
Epoch 116 Reward -65
Epoch 1272 Reward -2814
Epoch 708 Reward -2

Epoch 4252 Reward -5218
Epoch 1365 Reward -3609
Epoch 745 Reward -1198
Epoch 976 Reward -1726
Epoch 2724 Reward -3843
Epoch 7496 Reward -9641
Epoch 1129 Reward -1834
Epoch 1431 Reward -2244
Epoch 787 Reward -799
Epoch 2161 Reward -4036
Epoch 1333 Reward -2596
Epoch 909 Reward -948
Epoch 226 Reward -670
Epoch 2431 Reward -3217
Epoch 198 Reward -1209
Epoch 1280 Reward -1535
Epoch 158 Reward -171
Epoch 223 Reward -919
Epoch 141 Reward -405
Epoch 385 Reward -1450
Epoch 2025 Reward -4440
Epoch 284 Reward -980
Epoch 890 Reward -2036
Epoch 720 Reward -1794
Epoch 3621 Reward -6180
Epoch 2970 Reward -3882
Epoch 6687 Reward -9291
Epoch 62 Reward 159
Epoch 812 Reward -1013
Epoch 1735 Reward -2701
Epoch 835 Reward -1180
Epoch 1896 Reward -2520
Epoch 1090 Reward -1768
Epoch 181 Reward -211
Epoch 114 Reward 117
Epoch 791 Reward -992
Epoch 562 Reward -619
Epoch 678 Reward -1644
Epoch 303 Reward -261
Epoch 840 Reward -1356
Epoch 485 Reward -2045
Epoch 88 Reward -280
Epoch 1594 Reward -3244
Epoch 3750 

In [13]:
# tensorflow와 그 안의 모델을 실행
with tf.Session() as sess:
    # 저장 객체 생성
    saver = tf.train.Saver()
    
    # 저장된 모델 로드
    saver.restore(sess, './model-1306635') 

    done = False
    obs = env.reset()
    total_reward = 0


    #상태가 최종 상태가 아닌 동안
    while not done:

        display.clear_output(wait=True)
        display.display(plt.gcf())
        env.render()

        # 전처리된 게임 화면 가져오기
        obs = env.render('dqn')

        # 게임 화면을 피드하고 각 작업에 대한 Q 값을 가져오기
        actions = mainQ_outputs.eval(feed_dict={X:[obs], in_training_mode: False})

        # 행동 가져오기
        action = np.argmax(actions, axis =-1)

        #행동을 수행하고 다음 상태인 next_obs로 이동하여 보상을 받는다.
        next_obs, reward, done, _ = env.step(action)
        total_reward += reward
        print("Episode Reward= ", total_reward)
        time.sleep(0.5)


o - - - - - - - - - - - - - - - - - o
|   |       |           |           |
|           | - |   |   | - | - |   |
|   |               |   |           |
| - |                       | - |   |
|       |   |                       |
|   |     4                 | - |   |
|   |     4                 |       |
|   | - | - | - |           | - |   |
|                               | - |
o - - - - - - - - - - - - - - - - - o

Episode Reward=  -3961
